In [ ]:
!pip install soynlp -q

#Preprocess: text를 전처리하는 함수

import re
import json
from soynlp.normalizer import *

def PreProcess(text):
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char]) # 문제를 일으킬 수 있는 문자 제거(너비가 0인 공백문자,인코딩 정보를 알려주는 역할 문자)
    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "} #(표의 공백문자, 한칸, 엔터+스페이스, 빈 공백-유니코드에서 )
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])
    text = re.sub(pattern='Posted on [0-9]{4} [0-9]{2} [0-9]{2} .+ Posted in \S+ \s?', \
                  repl='', string=text) #정규 표현식 패턴과 일치하는 문자열을 찾아 다른 문자열로 대체 (년도)
    text = re.sub(pattern='Posted on [0-9]{8} .+ Posted in \S+ \s?', \
                  repl='', string=text)
    text = re.sub(pattern='[0-9]{4}년 [0-9]{,2}월 [0-9]{,2}일 [0-9]{,2}시 [0-9]{,2}분 [0-9]{,2}초', \
                  repl='', string=text) #(년도 시간)
    text = re.sub(pattern='[0-9]{4}. [0-9]{,2}. [0-9]{,2}', \
                  repl='', string=text) #필요없는 부분 제거 (년도 제거)
    text = re.sub(pattern = '^[a-zA-Z0-9]+@[a-zA-Z0-9]+$', \
                  repl ='', string = text) #이메일 제거
    text = re.sub(pattern = '#\S+', \
                  repl ='', string = text) # "#문자" 형식 어절 제거
    text = re.sub(pattern = '@\w+', \
                  repl ='', string = text) # "@문자" 형식 어절 제거
    text = re.sub(pattern = '\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ', \
                  repl ='', string = text) # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = repeat_normalize(text, num_repeats=2).strip() # 중복 문자 처리
    _filter = re.compile('[ㄱ-ㅣ]+') 
    text = _filter.sub('', text) #자모음제거
    _filter = re.compile('[^가-힣 0-9 a-z A-Z \. \, \' \" \? \!]+') 
    text = _filter.sub('', text) #(".",",","'","?","!")를 제외한 특수문자 제거
    _filter = re.compile('[\n]')
    text = _filter.sub("", text) #개행제거
    return text

#save_data: 데이터를 합쳐서 저장하는 함수

from glob import glob

def save_data(file_path, save_path):
    print('{} data saving.'.format(file_path.split('/')[-1]))
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        text = json_data['train']
        length = len(text)
        data = json.dumps(text)
        data = data[9:-2].encode('utf-8')
        data = data.decode('unicode_escape')
        print(type(data))
        print(data)
        text = PreProcess(data)
        print(text)
        with open(save_path, 'w') as f:
                f.write(data)  #전처리 된 글 파일에 쓰기


    print("\nAll saved.".format(file_path.split('/')[-1]))


#합칠 데이터와 새로 저장할 데이터 Path 지정 => 전처리할 데이터와 전처리 저장할 데이터에 맞는 위치에 맞게 변경

file_1 = './essay.json' #크롤링 후 수집한 데이터의 위치(전처리 이전) => 팀 데이터 보관위치에 맞게 변경
file_2 = './Crawling/textcrawler/use_data/geulteen.txt' #크롤링 후 수집한 데이터의 위치(전처리 이전) => 팀 데이터 보관위치에 맞게 변경
save_path_1 = './data_1.txt' #저장할 데이터 위치(전처리된) => 팀 데이터 보관할 위치에 맞게 변경
save_path_2 = './Crawling/textcrawler/use_data/data_2.txt' #저장할 데이터 위치(전처리된) => 팀 데이터 보관할 위치에 맞게 변경

#함수 실행

save_data(file_1, save_path_1)
#save_data(file_2, save_path_2)


#저장된 데이터 확인 (굳이 필요하지 않음)

with open(save_path_1, 'r') as f:
    data = f.read()

data[:500]